# Libraries/Dependencies

In [83]:
import pandas as pd
import numpy as np
import datetime

## Load csv into pandas dataframe

In [2]:
dataframe = pd.read_csv('/Users/hparikh/ds-coding-interview-data-engineering/oppty_history.csv')

## Let's take a peak at our dataset

In [75]:
dataframe.head(10)

,opptyId,field,oldValue,newValue,updateTime
0,0,Stage,NaN,Qualify,2014-05-27T18:50:14
1,0,Forecast,NaN,Best Case,2014-05-27T18:50:14
2,0,created,NaN,NaN,2014-05-27T18:50:14
3,0,Amount,NaN,795.53,2014-06-17T18:54:00
4,0,Stage,Qualify,Closed - Won,2014-07-09T20:11:05
5,0,Forecast,Best Case,Closed,2014-07-09T20:11:05
6,1,Stage,NaN,Target,2013-08-13T04:36:09
7,1,Forecast,NaN,Omitted,2013-08-13T04:36:09
8,1,created,NaN,NaN,2013-08-13T04:36:09
9,1,Amount,NaN,395.0,2013-08-15T03:51:15


In [46]:
print("There are total rows: {0} and columns: {1}".format(dataframe.shape[0], dataframe.shape[1]))

There are total rows: 907012 and columns: 5


## How many deals did we close out of all? 

In [67]:
def getInsightsFromData(input_dataframe):
    total_num_of_deals = input_dataframe['opptyId'].nunique()
    num_of_closed_deals = input_dataframe[input_dataframe["newValue"] == 'Closed'].shape[0]
    
    percentage_of_deal_closing = num_of_closed_deals/total_num_of_deals
    
    return num_of_closed_deals, total_num_of_deals, percentage_of_deal_closing

num_of_closed_deals, total_num_of_deals, percentage_of_deal_closing = getInsightsFromData(dataframe)

print("Number of Closed deals are {0}/{1} = {2}%".format(num_of_closed_deals, total_num_of_deals, round(percentage_of_deal_closing, 2)))


Number of Closed deals are 78407/139726 = 0.56%


# Task1

#### Please compute the average lifetime of WON opportunities, in terms of median and mean, respectively.  Based on the results, do you think the hypothesis that “it takes 3 months to win one opportunity” to be true? 

In [44]:
def findLifeTimeOfOpportunities(input_dataframe):
    """
    This function takes dataframe as an input 
    and return mean and median lifetime of all opportunities.
    """
    
    # Let's start with getting close deals information.
    close_deals_info = input_dataframe[input_dataframe["newValue"] == 'Closed'][['opptyId', 'updateTime']]
    
    # Once we get information about which deals are closed then we can iterate 
    # one by one opportunity id and get create/start date of that. 
    rows_list = []
    for each_id in close_deals_info['opptyId'].to_list():
        deal_start_dict = {}
        deal_start_dict['opptyId'] = each_id
        deal_start_dict['startTime'] = input_dataframe[input_dataframe['opptyId'] == each_id].iloc[0]['updateTime']
        rows_list.append(deal_start_dict)
        
    create_deals_info = pd.DataFrame(rows_list)  
    
    deals_info = pd.merge(close_deals_info, create_deals_info, on='opptyId')
    
    # Now, we need to find difference between start time and close time. 
    deals_info['diff_days'] = pd.to_datetime(deals_info['updateTime']) - pd.to_datetime(deals_info['startTime'])
    deals_info['diff_days'] = deals_info['diff_days']/np.timedelta64(1,'D')
    
    # At the end, take the mean and median of the column
    mean_time = deals_info['diff_days'].mean()
    median_time = deals_info['diff_days'].median()
    
    return mean_time, median_time

In [45]:
mean_time, median_time = findLifeTimeOfOpportunities(dataframe)

In [73]:
print(" Median lifetime: {0} days\n Mean lifetime: {1} days".format(int(median_time), int(mean_time)))

 Median lifetime: 27 days
 Mean lifetime: 53 days


### Answer: So our hypothesis seems off. It takes around 2 months to close a deal on average or win an opportunity

# Task 2

#### Now, think of the data to be much larger, say billions of records for one org, and we are serving hundreds of orgs.  Each day, new data are coming as some opportunities get updated.
#### Now we need to support analytics in real time like the following queries: 
#### * Show the aggregate amount of opportunities at certain time
#### * Identify which opportunities moved from one particular stage to another specific stage from timestamp 1 to timestamp 2.
#### How would you store the data?  What kind of schema would you use?  Pros/Cons in your design? 

# Task 3

#### Can you build a model to predict how likely one opportunity will be closed as “Closed-WON”? 

#### Caveat: Problem 3 can take a lot of time.
#### The real world data will be similar to the sample data but with more fields of various types. So it is great to let you have a taste through the exercise.  

## Let's extract features out and assign labels to each opportunity.

In [ ]:
allOpprtunityIDs = dataframe['opptyId'].to_list()
row_list = []

for each_id in allOpprtunityIDs:
    # Make this sample dictionary for all deals
    # TODO: We can add more features later on..
    deals_dictn = {
        'opptyId': each_id, 
        'amount': 0,
        'days': 0,
        'num_of_stages': 0,
        'Label': 0
    }
    dealDataFrame = dataframe[dataframe['opptyId'] == each_id]
    
    # Get just number of stages would be helpful to determine.
    deals_dictn['num_of_stages'] = len(dealDataFrame)
    
    for index, row in dealDataFrame.iterrows():
        if row['field'] == 'created':
            creationDate = datetime.datetime.strptime(row['updateTime'], '%Y-%m-%dT%H:%M:%S')
        elif row['field'] == 'Amount':
            deals_dictn['amount'] = row['newValue']
        elif row['newValue'] == 'Closed - Lost':
            endDate = datetime.datetime.strptime(row['updateTime'], '%Y-%m-%dT%H:%M:%S')
            deals_dictn['Label'] = 0
        elif row['newValue'] == 'Closed - Won':
            endDate = datetime.datetime.strptime(row['updateTime'], '%Y-%m-%dT%H:%M:%S')
            deals_dictn['Label'] = 1
    
    # Number of days to get a lifetime of an opportunity
    days = (endDate - creationDate).days
    deals_dictn['days'] = days
    
    row_list.append(deals_dictn)


In [90]:
featurize_dataframe = pd.DataFrame(row_list)  

[{'opptyId': 0,
  'amount': '795.53',
  'days': 43,
  'num_of_stages': 6,
  'Label': 1}]

### I have extracted few features out but did not get a time to work on any ML model. We could start with Logistic Regression to get the benchmarking and later how it performs, we can make decision what kind of algorithm we could use. 